<a href="https://colab.research.google.com/github/mehmetandas/Data-Mining-Project/blob/master/Target_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 11.0 MB/s eta 0:00:00


In [ ]:
# Veri işleme
import pandas as pd
import numpy as np
import re
import ast
import json
from collections import defaultdict
from itertools import combinations

# Görselleştirme
import matplotlib.pyplot as plt
import seaborn as sns

# Sklearn - Ön işleme
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder, StandardScaler

# Sklearn - Modelleme
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, train_test_split, cross_val_score

# XGBoost
import xgboost as xgb

# PyTorch
import torch

# Optimizasyon
import optuna
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
!git clone https://github.com/Abmn0/emlak_dataset.git

fatal: destination path 'emlak_dataset' already exists and is not an empty directory.


In [ ]:
df = pd.read_csv("emlak_dataset/veri.csv")

In [ ]:
df = df.drop(index=range(2122, 2142))  # İngilizce satırlar şimdilik silindi

In [ ]:
# Retro Synthwave paleti ve global stil ayarları
def set_synthwave_palette(style="whitegrid", context="notebook", font_family="sans-serif"):
    palette = ['#f72585', '#b5179e', '#7209b7', '#560bad', '#480ca8',
               '#3a0ca3', '#3f37c9', '#4361ee', '#4895ef', '#4cc9f0']

    sns.set_palette(palette)
    sns.set_style(style)
    sns.set_context(context)

    # Matplotlib global ayarları
    plt.rcParams.update({
        'axes.titlepad': 20,
        'axes.titlesize': 14,
        'axes.labelsize': 12,
        'font.family': font_family,
        'figure.autolayout': True,
        'axes.edgecolor': '#3a0ca3',  # Koyu mor çerçeve
        'axes.facecolor': '#ffffff',  # Beyaz arka plan
        'figure.facecolor': '#ffffff',  # Beyaz figür arka planı
        'axes.labelcolor': '#3a0ca3',  # Koyu mor etiketler
        'axes.titlecolor': '#3a0ca3',  # Koyu mor başlık
        'xtick.color': '#3a0ca3',  # Koyu mor tick etiketleri
        'ytick.color': '#3a0ca3',
        'grid.color': '#4cc9f0',  # Açık mavi grid
        'grid.alpha': 0.5
    })

    return palette

# Global ayarları uygula
palette = set_synthwave_palette()

In [ ]:
# 1. Hiç değeri olmayan sütunları silen fonksiyon
def remove_empty_columns(df):
    empty_cols = [col for col in df.columns if df[col].isna().all()]

    df = df.drop(columns=empty_cols)
    return df

# 2. Tek çeşit değer içeren sütunları silen fonkisyon
def remove_single_value_columns(df):
    single_value_cols = []
    for col in df.columns:
        unique_values = df[col].dropna().unique()
        if len(unique_values) <= 1:
            single_value_cols.append(col)

    df = df.drop(columns=single_value_cols)

    #Değer varsa 1 yoksa 0
    df['housingComplex'] = df['housingComplex'].notnull().astype(int)

    return df

# 3. Telefon numaralarını işleyen fonksiyon
def process_phone_numbers(df):
    if 'descriptionPhoneNumbers' in df.columns:
        df['PhoneNumberPlusFlag'] = df['descriptionPhoneNumbers'].apply(lambda x: 1 if pd.notna(x) and '+' in str(x) else 0)
    return df

# 4. Belirtilen sütunları silen fonksiyon
def remove_columns(df):
    columns_to_drop = ['title','description','currency','contact','images','mapLocation','listingUpdatedDate','land','timeShareName'
                       'deposit','detailUrl','firmUser','category','videoUrl','featuringProducts','whatsAppNumber','virtualTour',
                       'realtyIdentificationNo','listingId.1','realtyId	no','descriptionPhoneNumbers', 'detailDescription','deposit','landRegisterName']
    existing_columns = [col for col in columns_to_drop if col in df.columns]

    df.drop(columns=existing_columns, inplace=True)
    return df

# 5. Attributes özelliğini sütunlara ayıran ve oluşan sütunlara eşik değer kontrolü sağlayan fonksiyon
def process_attributes(df, min_feature_count=100):
    feature_counts = defaultdict(int)
    row_features = {}

    # Özellikleri toplama
    for idx, attr_str in df['attributes'].items():
        features = set()
        if pd.isna(attr_str):
            row_features[idx] = features
            continue

        data = ast.literal_eval(attr_str) if isinstance(attr_str, str) else attr_str

        def collect_features(obj):
            if isinstance(obj, dict):
                if 'name' in obj:
                    feature = obj['name']
                    features.add(feature)
                    feature_counts[feature] += 1
                for v in obj.values():
                    collect_features(v)
            elif isinstance(obj, list):
                for item in obj:
                    collect_features(item)

        collect_features(data)
        row_features[idx] = features

    # Filtrelenmiş özellikleri seçme
    filtered_features = {f: f"attr_{f}" for f, count in feature_counts.items() if count >= min_feature_count}
    print(f"Toplam {len(feature_counts)} özellik bulundu, {len(filtered_features)} tanesi {min_feature_count} eşiğini geçti.")

    # Yeni sütunları oluşturma
    new_columns = pd.DataFrame(0, index=df.index, columns=filtered_features.values())
    new_columns['total_features'] = 0

    for idx, features in row_features.items():
        valid_features = [f for f in features if f in filtered_features]
        for feature in valid_features:
            new_columns.at[idx, filtered_features[feature]] = 1
        new_columns.at[idx, 'total_features'] = len(valid_features)

    # Dataframe'i birleştirme
    df = pd.concat([df.drop(columns=['attributes']), new_columns], axis=1)
    return df


# 6. firm sütununda 'inşaat' kelimesi geçenleri işaretleme
def process_firm(df):
    df['insaat_var_mi'] = df['firm'].fillna("").str.lower().apply(
        lambda x: 1 if any(kelime in x for kelime in ['inşaat', 'insaat', 'ınşaat']) else 0
    )
    return df

# 7. sqm sütununu kategorilere ayırıp one-hot encode etme ve eşik kontrolü
def process_sqm(df):
    def extract_sqm(json_str):
        data = json.loads(json_str.replace("'", '"'))  # JSON string ise düzelt
        net = data.get('netSqm', None)
        gross = data.get('grossSqm', None)

        # gross listeyse ilkini al
        if isinstance(gross, list):
            gross = gross[0] if gross else None

        return pd.Series({'net': net, 'gross': gross})

    extracted = df['sqm'].apply(extract_sqm)
    df = pd.concat([df, extracted], axis=1)

    return df

# 8. rental sütunu fonksiyonu
def process_rental_sub(df):
    def get_amount(json_str):
        if not isinstance(json_str, str):
            return None
        try:
            data = json.loads(json_str.replace("'", '"'))  # Tek tırnakları düzelt
            return data.get('amount', None)
        except Exception:
            return None

    df['rental_amount'] = df['rental'].apply(get_amount)
    df['subCategory'] = df['subCategory'].apply(lambda x: json.loads(x).get('typeName', None) if isinstance(x, str) else None)

    return df

# 9. redirectLink fosnkiyonu
def process_redirectLink(df):

    def extract_subcategory(link):
        match = re.search(r'"linkedSubcategoryUrl":"(.*?)"', str(link))
        return match.group(1) if match else None

    # Alt kategori sütununu oluştur
    df['Subcategory'] = df['redirectLink'].apply(extract_subcategory)

    # One-hot encoding işlemi
    df = pd.get_dummies(df, columns=['Subcategory'], prefix='Subcat')

    return df

# 10. date ler için olan fonskiyon
def process_dates(df):
    # Tarih sütunlarını timezone-naive datetime formatına çevir
    df['createdDate'] = pd.to_datetime(df['createdDate'], utc=True).dt.tz_localize(None)
    df['updatedDate'] = pd.to_datetime(df['updatedDate'], utc=True).dt.tz_localize(None)

    # Sadece gün, ay ve yıl bilgilerini karşılaştır (saat kısmını at)
    df['sameDayUpdate'] = (df['createdDate'].dt.date == df['updatedDate'].dt.date).astype(int)

    # Sütun sırasını düzenle - sameDayUpdate sütununu createdDate ve updatedDate sütunlarının yanına taşı
    cols = list(df.columns)
    created_idx = cols.index('createdDate')
    updated_idx = cols.index('updatedDate')
    last_date_idx = max(created_idx, updated_idx)

    # sameDayUpdate sütununu son tarih sütunundan sonraki pozisyona yerleştir
    cols.remove('sameDayUpdate')
    cols.insert(last_date_idx + 1, 'sameDayUpdate')

    # Sütunları yeniden düzenle
    df = df[cols]

    return df

# 11. ilgili sütunlarda name-count değerlerini çekme ve önceki sütunları silme
def extract_values(df):
    def safe_eval(json_str):
        try:
            return ast.literal_eval(json_str)
        except Exception:
            return None  # Değerlendirilemeyenler için None döndür

    def extract_from_row(json_str, keys):
        data = safe_eval(json_str)

        if isinstance(data, dict):
            return {key: data.get(key, None) for key in keys}

        elif isinstance(data, list):
            extracted = {
                key: [item.get(key, None) for item in data if isinstance(item, dict)]
                for key in keys
            }

            for key in extracted:
                values = extracted[key]
                if all(isinstance(v, int) for v in values):
                    extracted[key] = ", ".join(map(str, values))
                elif all(isinstance(v, str) for v in values):
                    extracted[key] = ", ".join(values)
                else:
                    extracted[key] = ", ".join(map(str, values))

            return extracted

        return {key: None for key in keys}

    name_columns = ["areas", "district", "county", "residence", "heating",
                    "buildState", "usage", "credit", "barter"]

    for col in name_columns:
        extracted = df[col].astype(str).apply(lambda x: extract_from_row(x, ["name"]))
        df[f"{col}_name"] = extracted.apply(lambda x: x["name"])
        df.drop(columns=[col], inplace=True)

    extracted_floor = df["floor"].astype(str).apply(lambda x: extract_from_row(x, ["name", "count"]))
    df["floor_name"] = extracted_floor.apply(lambda x: x["name"])
    df["floor_count"] = extracted_floor.apply(lambda x: x["count"])
    df.drop(columns=["floor"], inplace=True)

    return df

# 12. Sides Sütunu İçin Bölgelere ayırma

def sides_process(df):
    df['sides'] = df['sides'].astype(str).str.lower()

    df['kuzey'] = df['sides'].apply(lambda x: 1 if 'kuzey' in x else 0)
    df['güney'] = df['sides'].apply(lambda x: 1 if 'güney' in x else 0)
    df['doğu']  = df['sides'].apply(lambda x: 1 if 'doğu'  in x else 0)
    df['batı']  = df['sides'].apply(lambda x: 1 if 'batı'  in x else 0)

    def extract_amount(fee):
        if pd.isna(fee):
            return 0.0
        fee_dict = ast.literal_eval(fee)
        return float(fee_dict.get('amount', 0))

    df['amount'] = df['fee'].apply(extract_amount)

    return df

# 13. İşlenmiş sütunları silen fonksiyon
def delete_columns(df):
    silinecek_sutunlar = [
        'realtyId', 'no', 'startDate', 'endDate', 'createdDate', 'updatedDate',
        'sqm', 'fuel', 'build', 'redirectLink', 'firm', 'fee', 'sides','rental'
    ]
    df.drop(columns=[col for col in silinecek_sutunlar if col in df.columns], inplace=True)
    return df

def fill_missing_values(df):

    # Sayısal sütunlar
    numerical_columns = ['age', 'searchScore', 'total_features', 'net', 'gross', 'amount', 'rental_amount']

    # Sayısal sütunlardaki eksik (NaN) veya 0 olan değerleri -999 ile doldur
    for col in numerical_columns:
        df[col] = df[col].apply(lambda x: -999 if pd.isna(x) or x == 0 else x)

    # Diğer tüm sütunlar (kategorik sütunlar)
    for col in df.columns:
        if col not in numerical_columns:
            df[col] = df[col].fillna('NaN').apply(lambda x: 'NaN' if pd.isna(x) else x)

    return df

# 14. Belirtilen sütunları encode etme ve oluşan sütunları belli bir eşik değerine göre eleme
def encode_columns(df, threshold=0.05):
    # Kopya oluştur (orijinal veriyi koru)
    df = df.copy()

    # SADECE encode edilecek sütunları string'e çevir
    encode_columns = ['room', 'livingRoom', 'bathRoom', 'floor_count',  # label_columns
                     'subCategory', 'roomAndLivingRoom', 'registerState',  # one_hot_columns
                     'areas_name', 'district_name', 'county_name', 'residence_name',
                     'heating_name', 'buildState_name', 'usage_name',
                     'credit_name', 'barter_name', 'floor_name',
                     'authorizedRealtor', 'furnished']

    # Sadece belirtilen sütunları string yap (diğerlerine dokunma)
    for col in encode_columns:
        if col in df.columns:
            df[col] = df[col].astype(str)

    # Label Encoding
    le = LabelEncoder()
    label_columns = ['room', 'livingRoom', 'bathRoom', 'floor_count']
    for col in label_columns:
        if col in df.columns:
            df[col] = le.fit_transform(df[col])

    # One-Hot Encoding
    one_hot_columns = ['subCategory', 'roomAndLivingRoom', 'registerState',
                      'areas_name', 'district_name', 'county_name', 'residence_name',
                      'heating_name', 'buildState_name', 'usage_name',
                      'credit_name', 'barter_name', 'floor_name',
                      'authorizedRealtor', 'furnished']

    df = pd.get_dummies(df, columns=one_hot_columns, drop_first=True)

    # Eşik değer kontrolü (SADECE one-hot ile oluşan yeni sütunlara uygula)
    original_columns = df.columns
    new_columns = [col for col in df.columns if col not in original_columns]
    for col in new_columns:
        value_counts = df[col].value_counts(normalize=True)
        if value_counts.iloc[0] <= threshold:
            df.drop(columns=[col], inplace=True)

    return df

# 15. Belirtilen sütunları scale etme
def scale_selected_columns(df):
    numeric_columns = ['age', 'searchScore', 'total_features', 'amount', 'rental_amount']

    # Min-Max Scaler'ı başlatıyoruz
    scaler = MinMaxScaler()

    # Sayısal sütunları ölçeklendiriyoruz
    df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

    return df

# 16. True/False string değerlerini bool veri tipine dönüştürme
def convert_to_bool(df):

    # Sadece object/string tipindeki sütunları al
    str_cols = df.select_dtypes(include='object').columns

    for col in str_cols:
        unique_vals = df[col].dropna().unique()
        # Sadece "True" ve "False" içeriyorsa
        if set(unique_vals).issubset({"True", "False"}):
            df[col] = df[col].map({"True": True, "False": False})

    return df

# Sütun isimlerini kontrol etme ve temizleme
def clean_column_names(df):
    # Sütun isimlerini string'e çevir ve geçersiz karakterleri değiştir
    df.columns = [str(col).replace('[', '_').replace(']', '_').replace('<', '_').replace('>', '_') for col in df.columns]
    return df

# 17. Tüm işlemleri birleştiren ana fonksiyon
def process_dataframe_complete(df):
    processed_df = df.copy() # Üst üste çalıştırdığımızda veri kaybı sorunu yaşamamak için kopyasında işlem yapıyoruz

    processed_df = remove_empty_columns(processed_df)
    processed_df = remove_single_value_columns(processed_df)
    processed_df = process_phone_numbers(processed_df)
    processed_df = remove_columns(processed_df)
    processed_df = process_attributes(processed_df, min_feature_count=100)
    processed_df = process_firm(processed_df)
    processed_df = process_redirectLink(processed_df)
    processed_df = extract_values(processed_df)
    processed_df = process_dates(processed_df)
    processed_df = process_sqm(processed_df)
    processed_df = sides_process(processed_df)
    processed_df = process_rental_sub(processed_df)
    processed_df = delete_columns(processed_df)
    processed_df = scale_selected_columns(processed_df)
    processed_df = fill_missing_values(processed_df)
    processed_df = encode_columns(processed_df, threshold=0.2)
    processed_df = convert_to_bool(processed_df)
    processed_df = clean_column_names(processed_df)

    return processed_df

# Tüm fonksiyonları çalıştır
processed_df = process_dataframe_complete(df)

#Bu sütunu koruyoruz daha sonra tahmin edeceğiz
room_col = processed_df['room']
housing_col = processed_df['housingComplex']

Toplam 246 özellik bulundu, 71 tanesi 100 eşiğini geçti.


In [ ]:
processed_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2140 entries, 0 to 2159
Columns: 545 entries, listingId to furnished_True
dtypes: bool(453), float64(5), int64(86), object(1)
memory usage: 2.5+ MB


ÖN İŞLEME SONRASI 545 SÜTUN İLE EĞİTİM

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef, roc_auc_score, classification_report
import xgboost as xgb
import optuna

def run_xgboost_model(
    optuna_enabled=False,
    test_size=100,
    n_splits=5,
    random_state=42,
    n_trials=10,
    output_excel='test_predictions.xlsx'
):

    # Hedef ve özellik sütunlarını belirleme
    target_column = 'housingComplex'
    if target_column not in processed_df.columns:
        raise ValueError("processed_df'de 'housingComplex' sütunu bulunamadı.")

    # Özellik sütunlarını belirle, 'listingId' sütununu hariç tut
    feature_columns = [col for col in processed_df.columns if col not in [target_column, 'listingId']]

    # Özellik sütunlarının veri tiplerini kontrol et
    invalid_columns = []
    for col in feature_columns:
        if processed_df[col].dtype not in ['int64', 'float64', 'bool', 'category']:
            invalid_columns.append(col)
    if invalid_columns:
        raise ValueError(
            f"Geçersiz veri tipleri tespit edildi. XGBoost sadece int, float, bool veya category türlerini destekler. "
            f"Geçersiz sütunlar: {invalid_columns}"
        )

    # Sınıf frekanslarını kontrol et
    class_counts = processed_df[target_column].value_counts()
    print("Sınıf frekansları:\n", class_counts)

    # Veri setini ayırma
    train_df, test_df = train_test_split(
        processed_df, test_size=test_size, random_state=random_state, stratify=processed_df[target_column]
    )

    # Eğitim ve test verileri
    X = train_df[feature_columns]
    y = train_df[target_column]
    X_test_final = test_df[feature_columns]
    y_test_final = test_df[target_column]

    # scale_pos_weight hesaplama
    scale_pos_weight = sum(y == 0) / sum(y == 1)
    print(f"scale_pos_weight: {scale_pos_weight:.4f}")

    # Varsayılan model parametreleri
    default_params = {
        'objective': 'binary:logistic',
        'eta': 0.1,
        'max_depth': 6,
        'n_estimators': 1000,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'min_child_weight': 1,
        'gamma': 0,
        'reg_alpha': 0,
        'reg_lambda': 1,
        'random_state': random_state,
        'eval_metric': 'auc',
        'early_stopping_rounds': 30,
        'scale_pos_weight': scale_pos_weight
    }

    # Optuna ile hiperparametre optimizasyonu
    if optuna_enabled:
        def objective(trial):
            optuna_params = {
                'eta': trial.suggest_float('eta', 0.01, 0.3, log=True),
                'max_depth': trial.suggest_int('max_depth', 3, 10),
                'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
                'subsample': trial.suggest_float('subsample', 0.5, 1.0),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
                'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
                'gamma': trial.suggest_float('gamma', 0, 5),
                'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
                'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
                'objective': 'binary:logistic',
                'random_state': random_state,
                'eval_metric': 'auc',
                'early_stopping_rounds': 30,
                'scale_pos_weight': scale_pos_weight
            }

            # Çapraz doğrulama
            kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
            f1_scores = []

            for train_index, test_index in kf.split(X):
                X_train, X_test = X.iloc[train_index], X.iloc[test_index]
                y_train, y_test = y.iloc[train_index], y.iloc[test_index]

                X_train_split, X_val, y_train_split, y_val = train_test_split(
                    X_train, y_train, test_size=0.2, random_state=random_state, stratify=y_train
                )

                model = xgb.XGBClassifier(**optuna_params)
                model.fit(
                    X_train_split,
                    y_train_split,
                    eval_set=[(X_val, y_val)],
                    verbose=False
                )

                y_pred = model.predict(X_test)  # F1 skoru için sınıf tahminleri
                f1_scores.append(f1_score(y_test, y_pred))

            return np.mean(f1_scores)

        # Optuna çalıştır
        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=n_trials)
        best_params = study.best_params
        best_params.update({
            'objective': 'binary:logistic',
            'random_state': random_state,
            'eval_metric': 'auc',
            'early_stopping_rounds': 30,
            'scale_pos_weight': scale_pos_weight
        })
        print("En iyi parametreler:", best_params)
        model_params = best_params
    else:
        model_params = default_params

    # Çapraz doğrulama
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    accuracy_scores, f1_scores, mcc_scores, auc_scores = [], [], [], []

    for fold, (train_index, test_index) in enumerate(kf.split(X), 1):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Eğitim ve doğrulama setleri
        X_train_split, X_val, y_train_split, y_val = train_test_split(
            X_train, y_train, test_size=0.2, random_state=random_state, stratify=y_train
        )

        # Model eğitimi
        xgb_model = xgb.XGBClassifier(**model_params)
        xgb_model.fit(
            X_train_split,
            y_train_split,
            eval_set=[(X_val, y_val)],
            verbose=False
        )

        # Tahmin
        y_pred = xgb_model.predict(X_test)
        y_pred_proba = xgb_model.predict_proba(X_test)[:, 1]

        # Metrikler
        accuracy_scores.append(accuracy_score(y_test, y_pred))
        f1_scores.append(f1_score(y_test, y_pred))
        mcc_scores.append(matthews_corrcoef(y_test, y_pred))
        auc_scores.append(roc_auc_score(y_test, y_pred_proba))

        # Kat sonuçları
        print(f"Fold {fold}:")
        print(f"  Doğruluk: {accuracy_scores[-1]:.4f}")
        print(f"  F1 Skoru: {f1_scores[-1]:.4f}")
        print(f"  MCC: {mcc_scores[-1]:.4f}")
        print(f"  AUC-ROC: {auc_scores[-1]:.4f}")
        print("-" * 50)

    # Ortalama metrikler
    print("5 Katlı Çapraz Doğrulama Ortalama Sonuçları:")
    print(f"Ortalama Doğruluk: {np.mean(accuracy_scores):.4f} ± {np.std(accuracy_scores):.4f}")
    print(f"Ortalama F1 Skoru: {np.mean(f1_scores):.4f} ± {np.std(f1_scores):.4f}")
    print(f"Ortalama MCC: {np.mean(mcc_scores):.4f} ± {np.std(mcc_scores):.4f}")
    print(f"Ortalama AUC-ROC: {np.mean(auc_scores):.4f} ± {np.std(auc_scores):.4f}")

    # Nihai model
    xgb_model_final = xgb.XGBClassifier(**model_params)
    xgb_model_final.fit(X, y, eval_set=[(X, y)], verbose=False)

    # Nihai test tahminleri
    y_pred_final = xgb_model_final.predict(X_test_final)
    y_pred_proba_final = xgb_model_final.predict_proba(X_test_final)[:, 1]

    # Nihai metrikler
    print("\nAyrılan Test Seti Üzerinde Nihai Sonuçlar:")
    print(f"Doğruluk: {accuracy_score(y_test_final, y_pred_final):.4f}")
    print(f"F1 Skoru: {f1_score(y_test_final, y_pred_final):.4f}")
    print(f"MCC: {matthews_corrcoef(y_test_final, y_pred_final):.4f}")
    print(f"AUC-ROC: {roc_auc_score(y_test_final, y_pred_proba_final):.4f}")

    # Sınıflandırma raporu
    print("\nSınıflandırma Raporu (Test Seti):")
    print(classification_report(y_test_final, y_pred_final))

    # Tahminleri kaydet
    results_df = pd.DataFrame({
        f'Gerçek {target_column}': y_test_final,
        f'Tahmin Edilen {target_column}': y_pred_final
    })
    results_df.to_excel(output_excel, index=False)
    print(f"\nTahminler ve gerçek değerler '{output_excel}' dosyasına kaydedildi.")

    return results_df

OPTUNA KAPALI DEFAULT PARAMETRELERLE

In [ ]:
result_xgboost = run_xgboost_model(optuna_enabled=False)

Sınıf frekansları:
 housingComplex
0    1748
1     392
Name: count, dtype: int64
scale_pos_weight: 4.4545
Fold 1:
  Doğruluk: 0.8137
  F1 Skoru: 0.5422
  MCC: 0.4287
  AUC-ROC: 0.8614
--------------------------------------------------
Fold 2:
  Doğruluk: 0.8015
  F1 Skoru: 0.5318
  MCC: 0.4105
  AUC-ROC: 0.8486
--------------------------------------------------
Fold 3:
  Doğruluk: 0.8260
  F1 Skoru: 0.5590
  MCC: 0.4606
  AUC-ROC: 0.8720
--------------------------------------------------
Fold 4:
  Doğruluk: 0.8431
  F1 Skoru: 0.6049
  MCC: 0.5071
  AUC-ROC: 0.8997
--------------------------------------------------
Fold 5:
  Doğruluk: 0.8284
  F1 Skoru: 0.5783
  MCC: 0.4832
  AUC-ROC: 0.8609
--------------------------------------------------
5 Katlı Çapraz Doğrulama Ortalama Sonuçları:
Ortalama Doğruluk: 0.8225 ± 0.0141
Ortalama F1 Skoru: 0.5632 ± 0.0261
Ortalama MCC: 0.4580 ± 0.0351
Ortalama AUC-ROC: 0.8685 ± 0.0173

Ayrılan Test Seti Üzerinde Nihai Sonuçlar:
Doğruluk: 0.8500
F1 Skoru:

OPTUNA AÇIK HİPER PARAMETRE OPTİMİZASYONU İLE

In [ ]:
result_xgboost = run_xgboost_model(optuna_enabled=True)

[I 2025-05-07 07:13:33,411] A new study created in memory with name: no-name-41ecb454-5795-474b-a13c-4f249769d5bc


Sınıf frekansları:
 housingComplex
0    1748
1     392
Name: count, dtype: int64
scale_pos_weight: 4.4545


[I 2025-05-07 07:13:43,757] Trial 0 finished with value: 0.5701222775354255 and parameters: {'eta': 0.02748685173861914, 'max_depth': 6, 'n_estimators': 852, 'subsample': 0.6342722720359109, 'colsample_bytree': 0.6694338912056643, 'min_child_weight': 3, 'gamma': 2.6394148445433534, 'reg_alpha': 0.2402487729649172, 'reg_lambda': 2.6386349653342247}. Best is trial 0 with value: 0.5701222775354255.
[I 2025-05-07 07:13:53,312] Trial 1 finished with value: 0.5520806267383115 and parameters: {'eta': 0.06910048301340528, 'max_depth': 6, 'n_estimators': 567, 'subsample': 0.7262157111134264, 'colsample_bytree': 0.607258524141215, 'min_child_weight': 10, 'gamma': 4.560988598463877, 'reg_alpha': 9.613502347241676, 'reg_lambda': 3.4143123154955637}. Best is trial 0 with value: 0.5701222775354255.
[I 2025-05-07 07:13:59,383] Trial 2 finished with value: 0.5689068431278167 and parameters: {'eta': 0.13469044716396217, 'max_depth': 6, 'n_estimators': 810, 'subsample': 0.9675075368483225, 'colsample_by

En iyi parametreler: {'eta': 0.213946795254392, 'max_depth': 9, 'n_estimators': 603, 'subsample': 0.9787661329142636, 'colsample_bytree': 0.6249252087217333, 'min_child_weight': 2, 'gamma': 2.361839334144085, 'reg_alpha': 0.47947142529328746, 'reg_lambda': 9.704630762715713, 'objective': 'binary:logistic', 'random_state': 42, 'eval_metric': 'auc', 'early_stopping_rounds': 30, 'scale_pos_weight': 4.454545454545454}
Fold 1:
  Doğruluk: 0.8211
  F1 Skoru: 0.5731
  MCC: 0.4663
  AUC-ROC: 0.8779
--------------------------------------------------
Fold 2:
  Doğruluk: 0.7868
  F1 Skoru: 0.5672
  MCC: 0.4548
  AUC-ROC: 0.8348
--------------------------------------------------
Fold 3:
  Doğruluk: 0.8407
  F1 Skoru: 0.5806
  MCC: 0.4882
  AUC-ROC: 0.8675
--------------------------------------------------
Fold 4:
  Doğruluk: 0.8284
  F1 Skoru: 0.6196
  MCC: 0.5170
  AUC-ROC: 0.8922
--------------------------------------------------
Fold 5:
  Doğruluk: 0.8162
  F1 Skoru: 0.5714
  MCC: 0.4756
  AUC-

RFE İLE ÖZELLİK SEÇİMİ

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import r2_score
from itertools import combinations
import matplotlib.pyplot as plt
import torch

# RFE Fonksiyonu (numeric_columns korunuyor)
def run_rfe_separately(target_column='price', feature_exclude_columns=['listingId'], protected_columns=None,
                       model_params=None, cv=5, step=20, min_features_to_select=70, test_size=0.2,
                       random_state=42, min_steps=30):
    global processed_df
    if protected_columns is None:
        protected_columns = []

    # Özellikleri seç (hedef ve hariç tutulacak sütunlar hariç)
    features = [col for col in processed_df.columns if col != target_column and col not in feature_exclude_columns]
    if not features:
        raise ValueError("Seçilecek özellik bulunamadı. Veri setini kontrol edin.")

    # protected_columns'ın veri setinde mevcut olduğunu kontrol et
    protected_columns = [col for col in protected_columns if col in features]
    print(f"Korunacak sütunlar: {protected_columns}")

    X = processed_df[features]
    y = processed_df[target_column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Model parametreleri
    if model_params is None:
        model_params = {
            'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'random_state': random_state,
            'early_stopping_rounds': 60, 'eta': 0.024, 'max_depth': 5, 'n_estimators': 850,
            'subsample': 0.62, 'colsample_bytree': 0.86, 'min_child_weight': 1,
            'gamma': 3.5, 'reg_alpha': 7.9, 'reg_lambda': 0.15,
            'n_jobs': -1 if not torch.cuda.is_available() else 1, 'verbosity': 0,
            'tree_method': 'hist', 'device': 'cuda' if torch.cuda.is_available() else 'cpu'
        }

    # İlk model eğitimi
    model = xgb.XGBRegressor(**model_params)
    model.fit(X_train, y_train, eval_set=[(X_train, y_train)], verbose=False)

    # Düşük önemli özellikleri çıkar (protected_columns hariç)
    importance = pd.DataFrame({'Feature': X_train.columns, 'Importance': model.feature_importances_})
    low_importance_features = importance.sort_values('Importance')[:int(0.3 * len(X_train.columns))]['Feature'].tolist()
    low_importance_features = [f for f in low_importance_features if f not in protected_columns]
    X_train = X_train.drop(columns=low_importance_features)
    X_test = X_test.drop(columns=low_importance_features)

    n_features = X_train.shape[1]
    remaining_features = list(X_train.columns)
    r2_scores = []
    selected_features_history = []
    step_count = 0

    print(f"Başlangıç Özellik Sayısı: {n_features}\nRFE Başlıyor...")

    # RFE döngüsü
    while n_features > min_features_to_select and step_count < min_steps:
        print(f"{n_features} özellik ile eğitim...")
        X_train_sub = X_train[remaining_features]
        kf = KFold(n_splits=cv, shuffle=True, random_state=random_state)
        fold_r2 = []

        for train_idx, val_idx in kf.split(X_train_sub):
            X_tr, X_val = X_train_sub.iloc[train_idx], X_train_sub.iloc[val_idx]
            y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
            model = xgb.XGBRegressor(**model_params)
            model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=False)
            fold_r2.append(r2_score(y_val, model.predict(X_val)))

        avg_r2 = np.mean(fold_r2)
        r2_scores.append(avg_r2)
        selected_features_history.append(remaining_features.copy())
        print(f"Ortalama R²: {avg_r2:.4f}")

        model.fit(X_train_sub, y_train, eval_set=[(X_train_sub, y_train)], verbose=False)
        importance = pd.DataFrame({'Feature': remaining_features, 'Importance': model.feature_importances_})
        removed_features = importance.sort_values('Importance')[:min(step, len(importance))]['Feature'].tolist()
        # protected_columns'ı koru
        removed_features = [f for f in removed_features if f not in protected_columns]
        remaining_features = [f for f in remaining_features if f not in removed_features]
        n_features = len(remaining_features)
        step_count += 1

    best_features = remaining_features
    best_score = r2_scores[-1] if r2_scores else 0

    print(f"\nRFE Tamamlandı.\nSon R²: {best_score:.4f}\nÖzellik sayısı: {len(best_features)}")

    # Final model eğitimi ve test
    final_model = xgb.XGBRegressor(**model_params)
    final_model.fit(X_train[best_features], y_train, eval_set=[(X_train[best_features], y_train)], verbose=False)
    test_preds = final_model.predict(X_test[best_features])
    test_r2 = r2_score(y_test, test_preds)
    print(f"Test R²: {test_r2:.4f}")

    # Özellik önemlerini görselleştir ve kaydet
    importance = pd.DataFrame({'Feature': best_features, 'Importance': final_model.feature_importances_})
    importance = importance.sort_values('Importance', ascending=False)
    plt.figure(figsize=(10, 8))
    importance.head(30).plot(kind='barh', x='Feature', y='Importance')
    plt.title('En Önemli 30 Özellik')
    plt.xlabel('Özellik Önemi')
    plt.tight_layout()
    plt.savefig('rfe_final_feature_importance.png')
    plt.close()

    importance.to_csv('rfe_final_feature_importance.csv', index=False)
    print(f"\nÖzellik önemleri 'rfe_final_feature_importance.csv' dosyasına kaydedildi.")

    # processed_df'yi güncelle
    keep_columns = best_features + [target_column] + feature_exclude_columns
    processed_df = processed_df[keep_columns]

    return best_features

# 1. RFE ile özellik seçimi (numeric_columns korunuyor)
numeric_columns = ['net', 'gross', 'room', 'bathRoom', 'housingComplex']
selected_features = run_rfe_separately(protected_columns=numeric_columns)

# 2. RFE sonrası processed_df durumu
print(f"RFE sonrası processed_df sütunları: {list(processed_df.columns)}")

Korunacak sütunlar: ['net', 'gross', 'room', 'bathRoom', 'housingComplex']
Başlangıç Özellik Sayısı: 381
RFE Başlıyor...
381 özellik ile eğitim...
Ortalama R²: 0.6138
361 özellik ile eğitim...
Ortalama R²: 0.5911
341 özellik ile eğitim...
Ortalama R²: 0.6022
321 özellik ile eğitim...
Ortalama R²: 0.6022
301 özellik ile eğitim...
Ortalama R²: 0.6071
281 özellik ile eğitim...
Ortalama R²: 0.6115
261 özellik ile eğitim...
Ortalama R²: 0.5972
241 özellik ile eğitim...
Ortalama R²: 0.6071
221 özellik ile eğitim...
Ortalama R²: 0.6039
201 özellik ile eğitim...
Ortalama R²: 0.6086
181 özellik ile eğitim...
Ortalama R²: 0.6066
161 özellik ile eğitim...
Ortalama R²: 0.5937
141 özellik ile eğitim...
Ortalama R²: 0.6092
121 özellik ile eğitim...
Ortalama R²: 0.6115
102 özellik ile eğitim...
Ortalama R²: 0.6140
83 özellik ile eğitim...
Ortalama R²: 0.6096

RFE Tamamlandı.
Son R²: 0.6096
Özellik sayısı: 64
Test R²: 0.5785

Özellik önemleri 'rfe_final_feature_importance.csv' dosyasına kaydedildi.
RF

<Figure size 1000x800 with 0 Axes>

ÖZELLİK MÜHENDİSLİĞİ

In [ ]:
# Özellik Mühendisliği Fonksiyonu
def create_engineered_features(df, numeric_cols=None, log_threshold=1e-6):
    df = df.copy()

    # Sayısal sütunları seç
    if numeric_cols is None:
        numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

    # numeric_cols'un df'de mevcut olduğunu kontrol et
    numeric_cols = [col for col in numeric_cols if col in df.columns]
    if not numeric_cols:
        raise ValueError("Hiçbir sayısal sütun bulunamadı veya belirtilen sütunlar veri setinde mevcut değil.")

    print(f"İşlenecek sayısal sütunlar ({len(numeric_cols)} adet): {numeric_cols}")

    new_features = {}

    # 2'li kombinasyonlar: çarpım, oran, toplam, fark
    for col1, col2 in combinations(numeric_cols, 2):
        new_features[f"{col1}_x_{col2}"] = df[col1] * df[col2]
        new_features[f"{col1}_div_{col2}"] = df[col1] / (df[col2] + log_threshold)
        new_features[f"{col1}_plus_{col2}"] = df[col1] + df[col2]
        new_features[f"{col1}_minus_{col2}"] = df[col1] - df[col2]

    # 3'lü kombinasyonlar: çarpım
    for col1, col2, col3 in combinations(numeric_cols, 3):
        new_features[f"{col1}_x_{col2}_x_{col3}"] = df[col1] * df[col2] * df[col3]

    # Tekil dönüşümler
    for col in numeric_cols:
        # Negatif veya sıfır değerler için kontrol
        new_features[f"log_{col}"] = np.log1p(df[col].clip(lower=0))
        new_features[f"sqrt_{col}"] = np.sqrt(df[col].clip(lower=0))
        mean = df[col].mean()
        std = df[col].std()
        new_features[f"zscore_{col}"] = (df[col] - mean) / (std + log_threshold)

    # Yeni özellikleri DataFrame'e dönüştür
    new_features_df = pd.DataFrame(new_features, index=df.index)

    # Ana DataFrame ile birleştir
    df = pd.concat([df, new_features_df], axis=1)

    print(f"Toplam yeni özellik sayısı: {len(new_features)}")
    return df

# 3. Özellik mühendisliği
processed_df = create_engineered_features(processed_df, numeric_cols=numeric_columns)

# Son durumu kontrol et
print(f"Son processed_df sütunları: {list(processed_df.columns)}")
print(f"Son processed_df şekli: {processed_df.shape}")

İşlenecek sayısal sütunlar (5 adet): ['net', 'gross', 'room', 'bathRoom', 'housingComplex']
Toplam yeni özellik sayısı: 65
Son processed_df sütunları: ['room', 'bathRoom', 'isMapHidden', 'age', 'housingComplex', 'isVideoTypeVimeo', 'stale', 'attr_Kablo TV-Uydu', "attr_E-5'e yakın", 'attr_Duşakabin', 'attr_Hidrofor', 'attr_Açık Yüzme Havuzu', 'attr_Kartonpiyer', 'attr_Deniz Ulaşımına yakın', 'attr_Giyinme Odası', 'attr_Vestiyer', 'attr_Su deposu', 'attr_Siding', 'Subcat_daire', 'Subcat_mustakil-ev', 'Subcat_villa', 'net', 'gross', 'subCategory_Bina', 'subCategory_Daire', 'subCategory_Villa', 'roomAndLivingRoom__"4"_', 'roomAndLivingRoom__"4+1"_', 'roomAndLivingRoom__"7+1"_', 'roomAndLivingRoom__"9 ve üzeri"_', 'registerState_Kat Mülkiyeti', 'registerState_NaN', 'areas_name_Dibektaş', 'areas_name_Diğer', 'areas_name_Diğer, Soğuksu', 'areas_name_Sapanca Merkez', 'areas_name_SapancaMerkez', 'areas_name_Serdivan Merkez', 'district_name_Arabacıalanı', 'district_name_Bıçkıatik', 'district_nam

SEÇİLEN VE EN İYİ ÖZELLİKLERİN TÜRETİLDİĞİ VERİ SETİNDE DEFAULT PARAMETRELERLE EĞİTİM

In [ ]:
result_xgboost = run_xgboost_model(optuna_enabled=False)

Sınıf frekansları:
 housingComplex
0    1748
1     392
Name: count, dtype: int64
scale_pos_weight: 4.4545
Fold 1:
  Doğruluk: 1.0000
  F1 Skoru: 1.0000
  MCC: 1.0000
  AUC-ROC: 1.0000
--------------------------------------------------
Fold 2:
  Doğruluk: 1.0000
  F1 Skoru: 1.0000
  MCC: 1.0000
  AUC-ROC: 1.0000
--------------------------------------------------
Fold 3:
  Doğruluk: 1.0000
  F1 Skoru: 1.0000
  MCC: 1.0000
  AUC-ROC: 1.0000
--------------------------------------------------
Fold 4:
  Doğruluk: 0.9975
  F1 Skoru: 0.9939
  MCC: 0.9924
  AUC-ROC: 0.9985
--------------------------------------------------
Fold 5:
  Doğruluk: 1.0000
  F1 Skoru: 1.0000
  MCC: 1.0000
  AUC-ROC: 1.0000
--------------------------------------------------
5 Katlı Çapraz Doğrulama Ortalama Sonuçları:
Ortalama Doğruluk: 0.9995 ± 0.0010
Ortalama F1 Skoru: 0.9988 ± 0.0025
Ortalama MCC: 0.9985 ± 0.0031
Ortalama AUC-ROC: 0.9997 ± 0.0006

Ayrılan Test Seti Üzerinde Nihai Sonuçlar:
Doğruluk: 1.0000
F1 Skoru:

SEÇİLEN VE EN İYİ ÖZELLİKLERİN TÜRETİLDİĞİ VERİ SETİNDE HİPER PARAMETRE OPTİMİZASYONU VE EĞİTİM

In [ ]:
result_xgboost = run_xgboost_model(optuna_enabled=True)

[I 2025-05-07 07:19:21,387] A new study created in memory with name: no-name-3c513260-c345-471e-a533-6fff3c636698


Sınıf frekansları:
 housingComplex
0    1748
1     392
Name: count, dtype: int64
scale_pos_weight: 4.4545


[I 2025-05-07 07:19:22,643] Trial 0 finished with value: 0.9987730061349694 and parameters: {'eta': 0.23063122454934556, 'max_depth': 3, 'n_estimators': 893, 'subsample': 0.9886790061791765, 'colsample_bytree': 0.5049188672306206, 'min_child_weight': 6, 'gamma': 3.5278907460658875, 'reg_alpha': 3.877080894744467, 'reg_lambda': 8.010280716619377}. Best is trial 0 with value: 0.9987730061349694.
[I 2025-05-07 07:19:23,900] Trial 1 finished with value: 0.9987730061349694 and parameters: {'eta': 0.2245156009258067, 'max_depth': 9, 'n_estimators': 539, 'subsample': 0.8567608665606143, 'colsample_bytree': 0.6546532645093701, 'min_child_weight': 10, 'gamma': 0.20726770525873295, 'reg_alpha': 3.9898542082771793, 'reg_lambda': 0.06912201141258567}. Best is trial 0 with value: 0.9987730061349694.
[I 2025-05-07 07:19:25,435] Trial 2 finished with value: 0.9987577639751553 and parameters: {'eta': 0.14751750275683365, 'max_depth': 10, 'n_estimators': 635, 'subsample': 0.6129898246186738, 'colsample

En iyi parametreler: {'eta': 0.23063122454934556, 'max_depth': 3, 'n_estimators': 893, 'subsample': 0.9886790061791765, 'colsample_bytree': 0.5049188672306206, 'min_child_weight': 6, 'gamma': 3.5278907460658875, 'reg_alpha': 3.877080894744467, 'reg_lambda': 8.010280716619377, 'objective': 'binary:logistic', 'random_state': 42, 'eval_metric': 'auc', 'early_stopping_rounds': 30, 'scale_pos_weight': 4.454545454545454}
Fold 1:
  Doğruluk: 1.0000
  F1 Skoru: 1.0000
  MCC: 1.0000
  AUC-ROC: 1.0000
--------------------------------------------------
Fold 2:
  Doğruluk: 1.0000
  F1 Skoru: 1.0000
  MCC: 1.0000
  AUC-ROC: 1.0000
--------------------------------------------------
Fold 3:
  Doğruluk: 1.0000
  F1 Skoru: 1.0000
  MCC: 1.0000
  AUC-ROC: 1.0000
--------------------------------------------------
Fold 4:
  Doğruluk: 0.9975
  F1 Skoru: 0.9939
  MCC: 0.9924
  AUC-ROC: 0.9985
--------------------------------------------------
Fold 5:
  Doğruluk: 1.0000
  F1 Skoru: 1.0000
  MCC: 1.0000
  AUC

In [ ]:
from sklearn.feature_selection import mutual_info_classif

X = processed_df.drop(columns=['housingComplex', 'listingId'])
y = processed_df['housingComplex']

mi = mutual_info_classif(X, y)
info_df = pd.DataFrame({'feature': X.columns, 'mutual_info': mi})
info_df.sort_values(by='mutual_info', ascending=False).head(10)

,feature,mutual_info
127,sqrt_housingComplex,0.476645
76,net_x_housingComplex,0.476411
88,gross_x_housingComplex,0.476411
106,net_x_gross_x_housingComplex,0.476411
108,net_x_room_x_housingComplex,0.476411
96,room_x_housingComplex,0.476411
89,gross_div_housingComplex,0.476411
126,log_housingComplex,0.476411
111,gross_x_room_x_housingComplex,0.476411
128,zscore_housingComplex,0.476411


In [ ]:
columns_to_remove = [
    'zscore_housingComplex',
    'net_x_housingComplex',
    'gross_x_housingComplex',
    'net_x_gross_x_housingComplex',
    'net_x_room_x_housingComplex',
    'room_x_housingComplex',
    'gross_div_housingComplex',
    'log_housingComplex',
    'gross_x_room_x_housingComplex',
    'sqrt_housingComplex'
]

filtered_df = processed_df.drop(columns=columns_to_remove, errors='ignore')

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef, roc_auc_score, classification_report
from sklearn.preprocessing import StandardScaler
import optuna
import numpy as np
import pandas as pd

def run_knn_model_realistic(
    df,
    target_column='housingComplex',
    test_ratio=0.2,
    random_state=42,
    n_splits=5,
    optuna_enabled=False,
    n_trials=20
):
    feature_columns = [col for col in df.columns if col not in [target_column, 'listingId']]

    # 1. Gerçek test setini ayır
    train_df, test_df = train_test_split(
        df, test_size=test_ratio, random_state=random_state, stratify=df[target_column]
    )

    X_train_full = train_df[feature_columns]
    y_train_full = train_df[target_column]
    X_test = test_df[feature_columns]
    y_test = test_df[target_column]

    # 2. Optuna ile en iyi parametreyi bul
    def objective(trial):
        n_neighbors = trial.suggest_int('n_neighbors', 3, 25)
        metric = trial.suggest_categorical('metric', ['euclidean', 'manhattan', 'minkowski'])
        f1s = []

        kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
        for train_idx, val_idx in kf.split(X_train_full):
            X_train, X_val = X_train_full.iloc[train_idx], X_train_full.iloc[val_idx]
            y_train, y_val = y_train_full.iloc[train_idx], y_train_full.iloc[val_idx]

            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            X_val_scaled = scaler.transform(X_val)

            model = KNeighborsClassifier(n_neighbors=n_neighbors, metric=metric)
            model.fit(X_train_scaled, y_train)
            y_pred = model.predict(X_val_scaled)
            f1s.append(f1_score(y_val, y_pred))

        return np.mean(f1s)

    if optuna_enabled:
        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=n_trials)
        best_params = study.best_params
        print("Best Optuna Parameters:", best_params)
    else:
        best_params = {'n_neighbors': 5, 'metric': 'minkowski'}

    # 3. KFold ile çapraz doğrulama skorları
    accs, f1s, mccs, aucs = [], [], [], []
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    for train_idx, val_idx in kf.split(X_train_full):
        X_train, X_val = X_train_full.iloc[train_idx], X_train_full.iloc[val_idx]
        y_train, y_val = y_train_full.iloc[train_idx], y_train_full.iloc[val_idx]

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_val_scaled = scaler.transform(X_val)

        model = KNeighborsClassifier(**best_params)
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_val_scaled)
        y_prob = model.predict_proba(X_val_scaled)[:, 1]

        accs.append(accuracy_score(y_val, y_pred))
        f1s.append(f1_score(y_val, y_pred))
        mccs.append(matthews_corrcoef(y_val, y_pred))
        aucs.append(roc_auc_score(y_val, y_prob))

    print("\nCross-Validation Results:")
    print(f"Accuracy: {np.mean(accs):.4f} ± {np.std(accs):.4f}")
    print(f"F1 Score: {np.mean(f1s):.4f} ± {np.std(f1s):.4f}")
    print(f"MCC: {np.mean(mccs):.4f} ± {np.std(mccs):.4f}")
    print(f"AUC-ROC: {np.mean(aucs):.4f} ± {np.std(aucs):.4f}")

    # 4. Gerçek test setiyle değerlendirme
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_full)
    X_test_scaled = scaler.transform(X_test)

    model = KNeighborsClassifier(**best_params)
    model.fit(X_train_scaled, y_train_full)
    y_pred_test = model.predict(X_test_scaled)
    y_prob_test = model.predict_proba(X_test_scaled)[:, 1]

    print("\nTest Set Results:")
    print(f"Accuracy: {accuracy_score(y_test, y_pred_test):.4f}")
    print(f"F1 Score: {f1_score(y_test, y_pred_test):.4f}")
    print(f"MCC: {matthews_corrcoef(y_test, y_pred_test):.4f}")
    print(f"AUC-ROC: {roc_auc_score(y_test, y_prob_test):.4f}")
    print(classification_report(y_test, y_pred_test))

    return {
        "cv_accuracy": np.mean(accs),
        "cv_f1": np.mean(f1s),
        "cv_mcc": np.mean(mccs),
        "cv_auc": np.mean(aucs),
        "test_accuracy": accuracy_score(y_test, y_pred_test),
        "test_f1": f1_score(y_test, y_pred_test),
        "test_mcc": matthews_corrcoef(y_test, y_pred_test),
        "test_auc": roc_auc_score(y_test, y_prob_test)
    }


In [ ]:
result = run_knn_model_realistic(df=filtered_df, optuna_enabled=True)


[I 2025-05-07 07:19:39,322] A new study created in memory with name: no-name-4035ff1b-ec42-442a-a252-e6ee46517cd5
[I 2025-05-07 07:19:39,555] Trial 0 finished with value: 0.8866432580508468 and parameters: {'n_neighbors': 7, 'metric': 'minkowski'}. Best is trial 0 with value: 0.8866432580508468.
[I 2025-05-07 07:19:39,775] Trial 1 finished with value: 0.8766435865019936 and parameters: {'n_neighbors': 11, 'metric': 'minkowski'}. Best is trial 0 with value: 0.8866432580508468.
[I 2025-05-07 07:19:40,022] Trial 2 finished with value: 0.8590623790525601 and parameters: {'n_neighbors': 8, 'metric': 'euclidean'}. Best is trial 0 with value: 0.8866432580508468.
[I 2025-05-07 07:19:40,247] Trial 3 finished with value: 0.8646396162500309 and parameters: {'n_neighbors': 23, 'metric': 'euclidean'}. Best is trial 0 with value: 0.8866432580508468.
[I 2025-05-07 07:19:40,468] Trial 4 finished with value: 0.8798789868630023 and parameters: {'n_neighbors': 3, 'metric': 'minkowski'}. Best is trial 0 w

Best Optuna Parameters: {'n_neighbors': 19, 'metric': 'manhattan'}

Cross-Validation Results:
Accuracy: 0.9901 ± 0.0057
F1 Score: 0.9726 ± 0.0152
MCC: 0.9672 ± 0.0182
AUC-ROC: 0.9999 ± 0.0001

Test Set Results:
Accuracy: 0.9836
F1 Score: 0.9530
MCC: 0.9447
AUC-ROC: 1.0000
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       350
           1       1.00      0.91      0.95        78

    accuracy                           0.98       428
   macro avg       0.99      0.96      0.97       428
weighted avg       0.98      0.98      0.98       428



In [ ]:
for k, v in result.items():
    print(f"{k}: {v:.4f}")

cv_accuracy: 0.9901
cv_f1: 0.9726
cv_mcc: 0.9672
cv_auc: 0.9999
test_accuracy: 0.9836
test_f1: 0.9530
test_mcc: 0.9447
test_auc: 1.0000


In [ ]:
correlations = processed_df.corr(numeric_only=True)['housingComplex'].abs().sort_values(ascending=False)
print(correlations.head(20))

log_housingComplex                   1.000000
zscore_housingComplex                1.000000
housingComplex                       1.000000
sqrt_housingComplex                  1.000000
room_x_housingComplex                0.969278
net_x_housingComplex                 0.879892
room_div_housingComplex              0.869001
gross_x_housingComplex               0.866248
net_x_room_x_housingComplex          0.778019
gross_x_room_x_housingComplex        0.761745
bathRoom_x_housingComplex            0.760259
room_x_bathRoom_x_housingComplex     0.676513
net_x_gross_x_housingComplex         0.599038
net_x_bathRoom_x_housingComplex      0.545484
gross_x_bathRoom_x_housingComplex    0.534276
bathRoom_div_housingComplex          0.484652
net_div_housingComplex               0.400737
room_minus_housingComplex            0.368422
bathRoom_minus_housingComplex        0.321867
attr_Açık Yüzme Havuzu               0.302733
Name: housingComplex, dtype: float64


In [ ]:
high_corr_cols = correlations[correlations > 0.40].index.tolist()
high_corr_cols.remove('housingComplex')  # hedef değişkeni sil

filtered_df_v2 = processed_df.drop(columns=high_corr_cols, errors='ignore')

In [ ]:
result = run_knn_model_realistic(df=filtered_df_v2, optuna_enabled=True)

[I 2025-05-07 07:19:50,059] A new study created in memory with name: no-name-8979441d-9769-4d6a-bfc9-5680d8be8b39
[I 2025-05-07 07:19:50,292] Trial 0 finished with value: 0.45837290297724265 and parameters: {'n_neighbors': 9, 'metric': 'minkowski'}. Best is trial 0 with value: 0.45837290297724265.
[I 2025-05-07 07:19:50,532] Trial 1 finished with value: 0.3057033028483387 and parameters: {'n_neighbors': 18, 'metric': 'euclidean'}. Best is trial 0 with value: 0.45837290297724265.
[I 2025-05-07 07:19:50,755] Trial 2 finished with value: 0.4523997784393824 and parameters: {'n_neighbors': 11, 'metric': 'minkowski'}. Best is trial 0 with value: 0.45837290297724265.
[I 2025-05-07 07:19:50,976] Trial 3 finished with value: 0.45639437695173657 and parameters: {'n_neighbors': 4, 'metric': 'minkowski'}. Best is trial 0 with value: 0.45837290297724265.
[I 2025-05-07 07:19:51,533] Trial 4 finished with value: 0.5284973067432865 and parameters: {'n_neighbors': 18, 'metric': 'manhattan'}. Best is tr

Best Optuna Parameters: {'n_neighbors': 3, 'metric': 'manhattan'}

Cross-Validation Results:
Accuracy: 0.9258 ± 0.0072
F1 Score: 0.7763 ± 0.0200
MCC: 0.7394 ± 0.0228
AUC-ROC: 0.9389 ± 0.0124

Test Set Results:
Accuracy: 0.9159
F1 Score: 0.7465
MCC: 0.7015
AUC-ROC: 0.9305
              precision    recall  f1-score   support

           0       0.93      0.97      0.95       350
           1       0.83      0.68      0.75        78

    accuracy                           0.92       428
   macro avg       0.88      0.82      0.85       428
weighted avg       0.91      0.92      0.91       428



In [ ]:
for k, v in result.items():
    print(f"{k}: {v:.4f}")

cv_accuracy: 0.9258
cv_f1: 0.7763
cv_mcc: 0.7394
cv_auc: 0.9389
test_accuracy: 0.9159
test_f1: 0.7465
test_mcc: 0.7015
test_auc: 0.9305
